# Preliminary Data Exploration

In [2]:
# Required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold

## Parse Data into Chunks

In [ ]:
# chunk_size = 5000  # rows per file
# for i, chunk in enumerate(pd.read_csv("./data/full_ddos_dataset.csv", chunksize=chunk_size)):
#     chunk.to_csv(f"./data/ddos_chunk_{i}.csv", index=False)

In [1]:
# Required libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold

# Load the data
data = pd.read_csv('./data/ddos_chunk_1294.csv')  # Chunk 1294 is the inflection point

# Normalize column names
data.columns = (
    data.columns
    .str.strip()
    .str.lower()
    .str.replace(r'[^\w]+', '_', regex=True)
)

# Normalize string values
for col in data.select_dtypes(include='object').columns:
    data[col] = data[col].str.strip().str.lower()

# # Drop `-inf/inf` rows `flow_byts_s` and `flow_pkts_s` # There are 14 in Chunk 1294
# data.replace([np.inf, -np.inf], np.nan, inplace=True)
# Drop rows where any value is +inf or -inf
data = data[~data.isin([np.inf, -np.inf]).any(axis=1)]

# Drop useless columns
data = data.drop(columns='unnamed_0')

# Investigating Low Variance Columns
numeric_cols = data.select_dtypes(include='number').columns
selector = VarianceThreshold(threshold=0.01)
selector.fit(data[numeric_cols])

low_variance = [col for col, var in zip(numeric_cols, selector.variances_) if var < 0.01]

# Drop low-variance columns
data = data.drop(columns=low_variance)

# Refresh numeric columns
numeric_cols = data.select_dtypes(include='number').columns